In [88]:
# pyarrow ou fastparquet
import pandas as pd
import numpy as np
from typing import Literal,Union
from collections import Counter
from numpy.random import default_rng
from pandas.api.types import is_datetime64_any_dtype, is_integer_dtype

In [2]:
full_history = pd.read_parquet('./data/full_history.parquet')

rastro_contratos = pd.read_parquet('./data/rastro_contratos.parquet')

In [3]:
display(full_history.head(3))
display(rastro_contratos.head(10))

,id_contrato,data_inicio_contrato,data_ref,safra,dias_atraso
0,10000000,2014-09-30,2015-01-01,201501,30
1,10000000,2014-09-30,2015-02-01,201501,15
2,10000000,2014-09-30,2015-03-01,201501,15


,id_antigo,id_novo,data_evento
0,10000003,10002500,2015-02-01
1,10000047,10002501,2015-02-01
2,10000048,10002502,2015-02-01
3,10000064,10002503,2015-02-01
4,10000081,10002504,2015-02-01
5,10000110,10002505,2015-02-01
6,10000164,10002506,2015-02-01
7,10000169,10002507,2015-02-01
8,10000170,10002508,2015-02-01
9,10000185,10002509,2015-02-01


### Verificação 1: IDs devem ser strings ou inteiros coerentes

In [4]:
assert full_history['id_contrato'].notna().all(), "❌ Existem contratos nulos em full_history"
assert rastro_contratos['id_antigo'].notna().all(), "❌ Existem IDs antigos nulos"
assert rastro_contratos['id_novo'].notna().all(), "❌ Existem IDs novos nulos"

#### Verifica se há conflitos de tipo (mistura de int/str)

In [5]:
# mapear tipos em full_history
tipos_ids_hist = full_history['id_contrato'].map(type).value_counts()

# mapear tipos em rastro_contratos
tipos_ids_rastro = pd.concat([
    rastro_contratos['id_antigo'].map(type),
    rastro_contratos['id_novo'].map(type)
]).value_counts()

print("Tipos encontrados em id_contrato:", tipos_ids_hist.to_dict())
print("Tipos encontrados em rastro_contratos:", tipos_ids_rastro.to_dict())

Tipos encontrados em id_contrato: {<class 'str'>: 115186}
Tipos encontrados em rastro_contratos: {<class 'str'>: 8246}


### Verificação 2: Datas devem estar em datetime64[ns]

In [6]:
assert pd.api.types.is_datetime64_any_dtype(full_history['data_ref']), "❌ 'data_ref' não está no formato datetime"

assert pd.api.types.is_datetime64_any_dtype(rastro_contratos['data_evento']), "❌ 'data_evento' não está no formato datetime"

### Verificação 3: Datas não devem ter valores faltantes

In [7]:
full_history.head(10)

,id_contrato,data_inicio_contrato,data_ref,safra,dias_atraso
0,10000000,2014-09-30,2015-01-01,201501,30
1,10000000,2014-09-30,2015-02-01,201501,15
2,10000000,2014-09-30,2015-03-01,201501,15
3,10000000,2014-09-30,2015-04-01,201501,30
4,10000000,2014-09-30,2015-05-01,201501,60
5,10000000,2014-09-30,2015-06-01,201501,90
6,10000000,2014-09-30,2015-07-01,201501,60
7,10000000,2014-09-30,2015-08-01,201501,90
8,10000000,2014-09-30,2015-09-01,201501,90
9,10000000,2014-09-30,2015-10-01,201501,60


In [8]:
assert full_history['data_ref'].notna().all(), "❌ Existem datas nulas em full_history['data_ref']"

assert rastro_contratos['data_evento'].notna().all(), "❌ Existem datas nulas em rastro_contratos['data_evento']"

### Verificação 4: Datas devem estar normalizadas (hora = 00:00:00)

In [9]:
# Para comparar datas corretamente, é fundamental que a parte de hora/minuto
# esteja "zerada", ou seja: datetime.time(0, 0). Isso evita erros sutis quando
# usamos comparações como: data_ref == data_evento

amostras_ref = full_history['data_ref'].dropna().dt.time.unique()
amostras_evt = rastro_contratos['data_evento'].dropna().dt.time.unique()

print("⏰ Horários únicos em data_ref:", amostras_ref)
print("⏰ Horários únicos em data_evento:", amostras_evt)

# Verificação de integridade das horas
hora_normal = pd.Timestamp("00:00:00").time()

ref_ok = all(t == hora_normal for t in amostras_ref)
evt_ok = all(t == hora_normal for t in amostras_evt)

if not ref_ok:
    print("⚠️ AVISO: 'data_ref' tem registros com horário diferente de 00:00:00")
if not evt_ok:
    print("⚠️ AVISO: 'data_evento' tem registros com horário diferente de 00:00:00")
if ref_ok and evt_ok:
    print("✅ Todas as datas estão normalizadas com horário 00:00:00 (seguro para comparação)")


⏰ Horários únicos em data_ref: [datetime.time(0, 0)]
⏰ Horários únicos em data_evento: [datetime.time(0, 0)]
✅ Todas as datas estão normalizadas com horário 00:00:00 (seguro para comparação)


# Testes de Continuidade

##### 1) se há algum contrato que salta no tempo

##### 2) construção da flag_acordo

##### 3) se algum contrato sofreu reaging

In [10]:
full_history.head()

,id_contrato,data_inicio_contrato,data_ref,safra,dias_atraso
0,10000000,2014-09-30,2015-01-01,201501,30
1,10000000,2014-09-30,2015-02-01,201501,15
2,10000000,2014-09-30,2015-03-01,201501,15
3,10000000,2014-09-30,2015-04-01,201501,30
4,10000000,2014-09-30,2015-05-01,201501,60


In [11]:
def diagnostico_descontinuidade(
    df: pd.DataFrame,
    *,
    col_id: str = "id_contrato",
    col_data: str = "data_ref",
    freq: Literal["M", "Q", "A", "D", "H"] = "M",
    verbose: bool = True
) -> dict:
    """
    Verifica descontinuidades na sequência de `col_data` para cada `col_id`.

    Retorna
    -------
    dict com:
      contratos_com_gap : set   IDs com ao menos um salto
      n_contratos_gap   : int   quantidade desses IDs
      proporcao_gap     : float porcentagem sobre o total
      distrib_gaps      : Counter {tamanho_gap: ocorrências}
      df_gaps           : DataFrame com detalhes (id, n_gaps, tam_max_gap)
    """
    # 1) preparo --------------------------------------------------------
    df_tmp = df[[col_id, col_data]].dropna().copy()
    df_tmp[col_data] = pd.to_datetime(df_tmp[col_data]).dt.normalize()
    df_tmp.sort_values([col_id, col_data], inplace=True)

    registros, distrib = [], Counter()

    # 2) loop por contrato ---------------------------------------------
    for cid, g in df_tmp.groupby(col_id):
        datas = g[col_data]

        if freq in {"M", "Q", "A"}:
            # Converte para Period -> inteiro -> diff
            num = datas.dt.to_period(freq).astype(int)
            saltos = num.diff().fillna(0).astype(int)
        elif freq == "D":
            saltos = (datas.diff().dt.days.fillna(0).astype(int))
        elif freq == "H":
            saltos = (datas.diff().dt.total_seconds()
                               .fillna(0)
                               .div(3600)
                               .astype(int))
        else:
            raise ValueError(f"freq '{freq}' não suportada")

        gaps = saltos[saltos > 1]

        if not gaps.empty:
            registros.append({
                "id_contrato": cid,
                "n_gaps": len(gaps),
                "tam_max_gap": gaps.max()
            })
            distrib.update(gaps.values)

    # 3) sumariza -------------------------------------------------------

    # 3) sumariza -----------------------------------------------------------
    if registros:                               # ↔ há pelo menos 1 gap
        df_gaps = pd.DataFrame(registros)
        contratos_com_gap = set(df_gaps[col_id])
    else:                                       # ↔ nenhum gap encontrado
        df_gaps = pd.DataFrame(columns=[col_id, "n_gaps", "tam_max_gap"])
        contratos_com_gap = set()

    n_contratos_gap = len(contratos_com_gap)
    total_contratos = df_tmp[col_id].nunique()
    proporcao_gap   = n_contratos_gap / total_contratos if total_contratos else 0

    n_contratos_gap   = len(contratos_com_gap)
    total_contratos   = df_tmp[col_id].nunique()
    proporcao_gap     = n_contratos_gap / total_contratos if total_contratos else 0

    resumo = {
        "contratos_com_gap": contratos_com_gap,
        "n_contratos_gap": n_contratos_gap,
        "proporcao_gap": proporcao_gap,
        "distrib_gaps": distrib,
        "df_gaps": df_gaps
    }

    # 4) prints de auditoria -------------------------------------------
    if verbose:
        print("📋 Diagnóstico de continuidade")
        print(f"• Contratos analisados ............: {total_contratos:,}")
        print(f"• Contratos com gap ...............: {n_contratos_gap:,} "
              f"({proporcao_gap:.1%})")
        if distrib:
            maior = max(distrib)
            print(f"• Maior gap encontrado ............: {maior} período(s)")
            print("• Top tamanhos de gap (freq):")
            for tam, freq_ in sorted(distrib.items(), key=lambda x: -x[1])[:10]:
                print(f"    – {tam} → {freq_:,}")
            print("• Exemplos de contratos problemáticos:")
            print(df_gaps.sort_values("tam_max_gap", ascending=False)
                        .head(5)
                        .to_string(index=False))
        else:
            print("✅ Nenhum salto detectado – dados contínuos")

    return resumo


In [12]:
res = diagnostico_descontinuidade(
    full_history,
    col_id="id_contrato",
    col_data="data_ref",
    freq="M")      # ou "D", "H"…

📋 Diagnóstico de continuidade
• Contratos analisados ............: 12,228
• Contratos com gap ...............: 0 (0.0%)
✅ Nenhum salto detectado – dados contínuos


In [13]:
full_history.columns.tolist()

['id_contrato', 'data_inicio_contrato', 'data_ref', 'safra', 'dias_atraso']

#### Criar gaps no histório para garantir confiabilidade na função anterior 

In [14]:
def inserir_gaps_aleatorios(
    df: pd.DataFrame,
    *,
    col_id: str = "id_contrato",
    col_data: str = "data_ref",
    frac_contratos: float = 0.20,      # % de contratos que receberão gap
    mean_gap: float = 2.0,             # média desejada (meses)
    max_gap: int = 5,                  # limite superior
    random_state: int | None = 42
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Remove linhas para criar gaps aleatórios em `data_ref`.

    Retorna
    -------
    df_gap      : DataFrame com registros faltantes simulados
    df_inj_info : DataFrame detalhando contrato, início e tamanho do gap
    """
    rng = default_rng(random_state)
    df_gap = df.copy()

    # 1. Seleciona aleatoriamente um subconjunto de contratos
    contratos = df[col_id].unique()
    n_select  = int(len(contratos) * frac_contratos)
    contratos_sel = rng.choice(contratos, size=n_select, replace=False)

    injec_log = []  # para registrar o que foi removido

    # 2. Processa cada contrato escolhido
    for cid in contratos_sel:
        g = df_gap.loc[df_gap[col_id] == cid].copy()
        datas = g[col_data].sort_values().dt.normalize().unique()

        if len(datas) < 8:          # pouco histórico? pula
            continue

        # 2a. Sorteia tamanho do gap (N~Normal, truncado entre 1 e max_gap)
        gap_len = int(
            np.clip(
                rng.normal(loc=mean_gap, scale=1.0),
                1, max_gap
            ).round()
        )

        # 2b. Escolhe ponto de início possível
        # (garante que existam gap_len meses consecutivos)
        idx_max_start = len(datas) - gap_len
        if idx_max_start <= 0:
            continue
        start_idx = rng.integers(0, idx_max_start)
        gap_start = datas[start_idx]

        # 2c. Constrói range mensal e remove
        gap_period = pd.date_range(gap_start, periods=gap_len, freq="MS")
        mask_remover = (df_gap[col_id] == cid) & (df_gap[col_data].isin(gap_period))
        df_gap = df_gap.loc[~mask_remover]

        injec_log.append({
            "id_contrato": cid,
            "gap_len": gap_len,
            "gap_start": gap_start
        })

    df_inj_info = pd.DataFrame(injec_log)
    return df_gap, df_inj_info


In [15]:
df_com_gaps, info = inserir_gaps_aleatorios(
    full_history,
    col_id="id_contrato",
    col_data="data_ref",
    frac_contratos=0.15,   # 15 % dos contratos afetados
    random_state=0
)

display(info.head())         # mostra onde e quanto foi removido

,id_contrato,gap_len,gap_start
0,10005898,1,2017-10-01
1,10005519,1,2016-09-01
2,10004460,3,2015-11-01
3,10005930,1,2016-06-01
4,10007484,3,2017-08-01


In [16]:
res = diagnostico_descontinuidade(
    df_com_gaps,
    col_id="id_contrato",
    col_data="data_ref",
    freq="M")      # ou "D", "H"…

📋 Diagnóstico de continuidade
• Contratos analisados ............: 12,228
• Contratos com gap ...............: 838 (6.9%)
• Maior gap encontrado ............: 6 período(s)
• Top tamanhos de gap (freq):
    – 3 → 301
    – 2 → 279
    – 4 → 198
    – 5 → 58
    – 6 → 2
• Exemplos de contratos problemáticos:
id_contrato  n_gaps  tam_max_gap
   10003600       1            6
   10006391       1            6
   10007533       1            5
   10007604       1            5
   10006213       1            5


In [17]:
df_com_gaps[df_com_gaps['id_contrato']=='10000140']

,id_contrato,data_inicio_contrato,data_ref,safra,dias_atraso
1319,10000140,2014-07-23,2015-01-01,201501,0
1320,10000140,2014-07-23,2015-02-01,201501,0
1323,10000140,2014-07-23,2015-05-01,201501,15
1324,10000140,2014-07-23,2015-06-01,201501,30
1325,10000140,2014-07-23,2015-07-01,201501,30
1326,10000140,2014-07-23,2015-08-01,201501,30
1327,10000140,2014-07-23,2015-09-01,201501,30
1328,10000140,2014-07-23,2015-10-01,201501,30
1329,10000140,2014-07-23,2015-11-01,201501,60


# Construção de flag_acordo

```mermaid
flowchart TD
    A[Inicio] --> B[Copiar dados e normalizar IDs e datas]
    B --> C[Construir dicionario de origem dos contratos]
    C --> D[Mapear id_original no historico]
    D --> E[Preparar eventos de renegociacao]
    E --> F[Mesclar eventos ao historico]
    F --> G[Criar flag_acordo para data exata do evento]
    G --> H[Reorganizar colunas para melhor leitura]
    H --> Z[Fim com DataFrame atualizado]

```

In [18]:
# cuidado com cópias de dataframes em ambiente de produção (custo de alocação de memória adicional)
# em ambiente de experimentação não há problema 
# cópia é bacana quando você quer evitar efeitos colaterais inesperados, ou quando segurança e legibilidade pesam mais do que performance

def criar_flag_acordo(
    full_history: pd.DataFrame,
    rastro_contratos: pd.DataFrame,
    *,
    col_id_hist: str = "id_contrato",       # coluna-ID no histórico
    col_id_antigo: str = "id_antigo",       # ID antigo no rastro
    col_id_novo: str = "id_novo",           # ID novo   no rastro
    col_evento: str = "data_evento",        # data em que o novo contrato nasce
    col_data_ref: str = "data_ref",         # data da linha no histórico
    verbose: bool = True
) -> pd.DataFrame:
    """
    Marca flag_acordo = 1 NO CONTRATO ANTIGO, no mês imediatamente
    anterior ao nascimento do contrato novo (data_evento – 1 mês).
    Também cria id_original (primeiro contrato da cadeia).

    Retorna uma cópia de full_history com:
        • id_original
        • data_evento  (replicada do rastro)
        • flag_acordo  (0/1)
    """
    # ── 1. Normalização ───────────────────────────────────────────────
    fh = full_history.copy()
    rc = rastro_contratos.copy()

    fh[col_id_hist]   = fh[col_id_hist].astype(str).str.strip()
    rc[col_id_antigo] = rc[col_id_antigo].astype(str).str.strip()
    rc[col_id_novo]   = rc[col_id_novo].astype(str).str.strip()

    fh[col_data_ref] = pd.to_datetime(fh[col_data_ref], errors="coerce")\
                           .dt.normalize()
    rc[col_evento]   = pd.to_datetime(rc[col_evento], errors="coerce")\
                           .dt.normalize()

    # ── 2. id_original (segue cadeia id_novo → id_antigo) ─────────────
    link = dict(zip(rc[col_id_novo], rc[col_id_antigo]))

    def _orig(cid: str) -> str:
        while cid in link:
            cid = link[cid]
        return cid

    fh["id_original"] = fh[col_id_hist].map(_orig)

    # ── 3. Flag de acordo no contrato antigo ──────────────────────────
    eventos = rc[[col_id_antigo, col_evento]].rename(
        columns={col_id_antigo: col_id_hist}
    )
    # mês em que a flag deve ser marcada = evento – 1 mês
    eventos["data_flag"] = eventos[col_evento] - pd.offsets.MonthBegin(1)

    # junta data_evento e data_flag ao histórico (apenas pelo contrato antigo)
    fh = fh.merge(eventos[[col_id_hist, col_evento, "data_flag"]],
                  how="left", on=col_id_hist)

    fh["flag_acordo"] = (fh[col_data_ref] == fh["data_flag"]).astype(int)
    fh.drop(columns="data_flag", inplace=True)      # coluna auxiliar

    # ── 4. Organização de colunas ─────────────────────────────────────
    base_cols = [
        "id_original",
        col_id_hist,
        "data_inicio_contrato",
        col_data_ref,
        "safra",
        "dias_atraso",
        col_evento,        # agora presente para auditoria
        "flag_acordo",
    ]
    outras = [c for c in fh.columns if c not in base_cols]
    fh = fh[base_cols + outras]

    # ── 5. Auditoria opcional ─────────────────────────────────────────
    if verbose:
        total_flags = int(fh["flag_acordo"].sum())
        contratos_flag = fh.loc[fh["flag_acordo"] == 1, col_id_hist].nunique()
        print("📌 criar_flag_acordo — auditoria")
        print(f"• Flags marcadas ...............: {total_flags:,}")
        print(f"• Contratos com flag_acordo=1 ..: {contratos_flag:,}")
        display(
            fh.loc[fh["flag_acordo"] == 1,
                   [col_id_hist, col_data_ref, "dias_atraso", col_evento]]
            .head()
        )

    return fh


In [19]:
df = criar_flag_acordo(
    full_history=full_history,
    col_id_hist='id_contrato',
    col_data_ref='data_ref',
    
    rastro_contratos=rastro_contratos,
    col_id_antigo='id_antigo',
    col_evento='data_evento'
    )

📌 criar_flag_acordo — auditoria
• Flags marcadas ...............: 4,123
• Contratos com flag_acordo=1 ..: 4,123


,id_contrato,data_ref,dias_atraso,data_evento
10,10000000,2015-11-01,60,2015-12-01
15,10000001,2015-05-01,90,2015-06-01
23,10000002,2015-08-01,90,2015-09-01
24,10000003,2015-01-01,120,2015-02-01
70,10000007,2015-04-01,90,2015-05-01


In [20]:
# verificação de shape (antes e após mesclagem)
full_history.shape, df.shape

((115186, 5), (115186, 8))

In [21]:
# verificação de fração de contratos com flag_acordo

df['flag_acordo'].value_counts(normalize=True, dropna=False)*100

flag_acordo
0    96.420572
1     3.579428
Name: proportion, dtype: float64

#### Coletando exemplo de contrato que recebeu flag_acordo = 1

In [22]:
df[df['flag_acordo']==1]['id_contrato'].sample(n=3,random_state=0).tolist()

['10003337', '10000061', '10007245']

In [23]:
df[df['id_contrato']=='10003337']

,id_original,id_contrato,data_inicio_contrato,data_ref,safra,dias_atraso,data_evento,flag_acordo
40398,10003337,10003337,2015-04-29,2015-05-01,201505,30,2016-11-01,0
40399,10003337,10003337,2015-04-29,2015-06-01,201505,30,2016-11-01,0
40400,10003337,10003337,2015-04-29,2015-07-01,201505,60,2016-11-01,0
40401,10003337,10003337,2015-04-29,2015-08-01,201505,60,2016-11-01,0
40402,10003337,10003337,2015-04-29,2015-09-01,201505,60,2016-11-01,0
40403,10003337,10003337,2015-04-29,2015-10-01,201505,60,2016-11-01,0
40404,10003337,10003337,2015-04-29,2015-11-01,201505,90,2016-11-01,0
40405,10003337,10003337,2015-04-29,2015-12-01,201505,60,2016-11-01,0
40406,10003337,10003337,2015-04-29,2016-01-01,201505,60,2016-11-01,0
40407,10003337,10003337,2015-04-29,2016-02-01,201505,60,2016-11-01,0


In [24]:
rastro_contratos[rastro_contratos['id_antigo']=='10003337']

,id_antigo,id_novo,data_evento
2343,10003337,10007348,2016-11-01


In [25]:
full_history[full_history['id_contrato'].isin(['10003337','10007348'])]

,id_contrato,data_inicio_contrato,data_ref,safra,dias_atraso
40398,10003337,2015-04-29,2015-05-01,201505,30
40399,10003337,2015-04-29,2015-06-01,201505,30
40400,10003337,2015-04-29,2015-07-01,201505,60
40401,10003337,2015-04-29,2015-08-01,201505,60
40402,10003337,2015-04-29,2015-09-01,201505,60
40403,10003337,2015-04-29,2015-10-01,201505,60
40404,10003337,2015-04-29,2015-11-01,201505,90
40405,10003337,2015-04-29,2015-12-01,201505,60
40406,10003337,2015-04-29,2016-01-01,201505,60
40407,10003337,2015-04-29,2016-02-01,201505,60


In [26]:
df[df['id_contrato'].isin(['10003337','10007348','10010483'])]

,id_original,id_contrato,data_inicio_contrato,data_ref,safra,dias_atraso,data_evento,flag_acordo
40398,10003337,10003337,2015-04-29,2015-05-01,201505,30,2016-11-01,0
40399,10003337,10003337,2015-04-29,2015-06-01,201505,30,2016-11-01,0
40400,10003337,10003337,2015-04-29,2015-07-01,201505,60,2016-11-01,0
40401,10003337,10003337,2015-04-29,2015-08-01,201505,60,2016-11-01,0
40402,10003337,10003337,2015-04-29,2015-09-01,201505,60,2016-11-01,0
40403,10003337,10003337,2015-04-29,2015-10-01,201505,60,2016-11-01,0
40404,10003337,10003337,2015-04-29,2015-11-01,201505,90,2016-11-01,0
40405,10003337,10003337,2015-04-29,2015-12-01,201505,60,2016-11-01,0
40406,10003337,10003337,2015-04-29,2016-01-01,201505,60,2016-11-01,0
40407,10003337,10003337,2015-04-29,2016-02-01,201505,60,2016-11-01,0


### Contratos sem nenhum acordo

In [27]:
# identificar os contratos que tiveram flag_acordo = 1
contratos_com_acordo = df.loc[df["flag_acordo"] == 1, "id_contrato"].unique()

# filtrar todos os que NÃO estão nessa lista
df_sem_acordo = df[~df["id_contrato"].isin(contratos_com_acordo)]

In [28]:
df_sem_acordo['id_contrato'].sample(n=3,random_state=0).tolist()

['10002238', '10003874', '10009286']

In [29]:
df[df['id_contrato']=='10002238']

,id_original,id_contrato,data_inicio_contrato,data_ref,safra,dias_atraso,data_evento,flag_acordo
26071,10002238,10002238,2014-10-03,2015-01-01,201501,60,NaT,0
26072,10002238,10002238,2014-10-03,2015-02-01,201501,60,NaT,0
26073,10002238,10002238,2014-10-03,2015-03-01,201501,60,NaT,0
26074,10002238,10002238,2014-10-03,2015-04-01,201501,30,NaT,0
26075,10002238,10002238,2014-10-03,2015-05-01,201501,30,NaT,0
26076,10002238,10002238,2014-10-03,2015-06-01,201501,30,NaT,0
26077,10002238,10002238,2014-10-03,2015-07-01,201501,30,NaT,0
26078,10002238,10002238,2014-10-03,2015-08-01,201501,30,NaT,0
26079,10002238,10002238,2014-10-03,2015-09-01,201501,30,NaT,0
26080,10002238,10002238,2014-10-03,2015-10-01,201501,30,NaT,0


In [30]:
df[df['id_contrato']=='10003874']

,id_original,id_contrato,data_inicio_contrato,data_ref,safra,dias_atraso,data_evento,flag_acordo
47123,10002848,10003874,2015-02-06,2015-08-01,201508,0,NaT,0
47124,10002848,10003874,2015-02-06,2015-09-01,201508,0,NaT,0
47125,10002848,10003874,2015-02-06,2015-10-01,201508,0,NaT,0
47126,10002848,10003874,2015-02-06,2015-11-01,201508,0,NaT,0
47127,10002848,10003874,2015-02-06,2015-12-01,201508,0,NaT,0
47128,10002848,10003874,2015-02-06,2016-01-01,201508,0,NaT,0
47129,10002848,10003874,2015-02-06,2016-02-01,201508,0,NaT,0
47130,10002848,10003874,2015-02-06,2016-03-01,201508,0,NaT,0
47131,10002848,10003874,2015-02-06,2016-04-01,201508,15,NaT,0
47132,10002848,10003874,2015-02-06,2016-05-01,201508,30,NaT,0


In [31]:
rastro_contratos[rastro_contratos['id_antigo']=='10003874']

,id_antigo,id_novo,data_evento


### Diagnóstico de Reaging

In [32]:
def diagnosticar_reaging(
    df: pd.DataFrame,
    *,
    col_orig: str = "id_original",
    col_id: str = "id_contrato",
    col_date: str = "data_ref",
    col_delay: str = "dias_atraso",
    col_flag: str = "flag_acordo",
    verbose: bool = True,
) -> dict:
    """
    Diagnostica re-aging em nível de cadeia (`id_original`).

    • Um *evento* de acordo é cada linha com flag_acordo = 1.  
    • Um evento é considerado re-aging quando:
        1. atraso_antes > 0  (até o mês da flag, inclusive) e
        2. atraso_depois = 0 (após o mês da flag).

    Classificação de cada cadeia
    ----------------------------
    - COM re-aging ........: possui ≥1 evento que atende aos critérios.
    - SEM re-aging ........: possui eventos, mas nenhum atende aos critérios.
    - COM *e* SEM re-aging : possui ≥1 evento re-aging **e** ≥1 evento não-re-aging.
    """
    # ── 1. Pré-processamento ───────────────────────────────────────────
    df = df[[col_orig, col_id, col_date, col_delay, col_flag]].copy()
    df[col_date] = pd.to_datetime(df[col_date]).dt.normalize()
    df.sort_values([col_orig, col_date], inplace=True)

    cadeias = df.loc[df[col_flag] == 1, col_orig].unique()
    stats = {
        "com_re": [],        # só re-aging
        "sem_re": [],        # só não-re-aging
        "mixed": []          # ambos
    }

    # ── 2. Avalia cada cadeia de contratos ────────────────────────────
    for cid in cadeias:
        g = df[df[col_orig] == cid]
        eventos = g[g[col_flag] == 1][col_date]

        reaging_flags = []
        for evento_dt in eventos:
            atraso_antes  = g[g[col_date] <= evento_dt][col_delay].max()
            atraso_depois = g[g[col_date] >  evento_dt][col_delay].min() \
                            if not g[g[col_date] >  evento_dt].empty else None
            reaging_flags.append((atraso_antes > 0) and (atraso_depois == 0))

        if all(reaging_flags):
            stats["com_re"].append(cid)
        elif not any(reaging_flags):
            stats["sem_re"].append(cid)
        else:
            stats["mixed"].append(cid)

    totais = {k: len(v) for k, v in stats.items()}
    total_cadeias = sum(totais.values())

    # ── 3. Seleciona cadeias-exemplo para exibição ─────────────────────
    def pick(df_base, ids):
        if not ids:
            return pd.DataFrame(), None
        # pega a primeira apenas para inspeção
        cid = ids[0]
        return df_base[df_base[col_orig] == cid], cid

    ex_com_re , id_com_re  = pick(df, stats["com_re"])
    ex_sem_re , id_sem_re  = pick(df, stats["sem_re"])
    ex_mixed  , id_mixed   = pick(df, stats["mixed"])

    # ── 4. Auditoria ──────────────────────────────────────────────────
    if verbose:
        print("📊 Re-aging por cadeia (id_original)")
        print(f"• Total de cadeias avaliadas ...........: {total_cadeias:,}")
        print(f"  ├─ COM re-aging ......................: {totais['com_re']:,}")
        print(f"  ├─ SEM re-aging ......................: {totais['sem_re']:,}")
        print(f"  └─ COM e SEM (mistas) ...............: {totais['mixed']:,}\n")

        if not ex_com_re.empty:
            print(f"🔍 Cadeia COM re-aging (id_original = {id_com_re})")
            display(ex_com_re[[col_orig, col_id, col_date,
                               col_delay, col_flag]])

        if not ex_sem_re.empty:
            print(f"\n🔍 Cadeia SEM re-aging (id_original = {id_sem_re})")
            display(ex_sem_re[[col_orig, col_id, col_date,
                               col_delay, col_flag]])

        if not ex_mixed.empty:
            print(f"\n🔍 Cadeia COM e SEM re-aging (id_original = {id_mixed})")
            display(ex_mixed[[col_orig, col_id, col_date,
                              col_delay, col_flag]])

    # ── 5. Retorno estruturado ────────────────────────────────────────
    return {
        "totais": totais,
        "exemplo_com":  ex_com_re,
        "exemplo_sem":  ex_sem_re,
        "exemplo_misto": ex_mixed
    }


In [33]:
resultado = diagnosticar_reaging(df)

📊 Re-aging por cadeia (id_original)
• Total de cadeias avaliadas ...........: 3,364
  ├─ COM re-aging ......................: 3,326
  ├─ SEM re-aging ......................: 30
  └─ COM e SEM (mistas) ...............: 8

🔍 Cadeia COM re-aging (id_original = 10000000)


,id_original,id_contrato,data_ref,dias_atraso,flag_acordo
0,10000000,10000000,2015-01-01,30,0
1,10000000,10000000,2015-02-01,15,0
2,10000000,10000000,2015-03-01,15,0
3,10000000,10000000,2015-04-01,30,0
4,10000000,10000000,2015-05-01,60,0
5,10000000,10000000,2015-06-01,90,0
6,10000000,10000000,2015-07-01,60,0
7,10000000,10000000,2015-08-01,90,0
8,10000000,10000000,2015-09-01,90,0
9,10000000,10000000,2015-10-01,60,0



🔍 Cadeia SEM re-aging (id_original = 10000446)


,id_original,id_contrato,data_ref,dias_atraso,flag_acordo
4279,10000446,10000446,2015-01-01,0,0
4280,10000446,10000446,2015-02-01,0,0
4281,10000446,10000446,2015-03-01,0,1
35209,10000446,10002926,2015-04-01,0,0



🔍 Cadeia COM e SEM re-aging (id_original = 10000271)


,id_original,id_contrato,data_ref,dias_atraso,flag_acordo
2650,10000271,10000271,2015-01-01,0,0
2651,10000271,10000271,2015-02-01,0,0
2652,10000271,10000271,2015-03-01,0,0
2653,10000271,10000271,2015-04-01,0,1
37814,10000271,10003137,2015-05-01,0,0
37815,10000271,10003137,2015-06-01,0,0
37816,10000271,10003137,2015-07-01,0,0
37817,10000271,10003137,2015-08-01,0,0
37818,10000271,10003137,2015-09-01,0,0
37819,10000271,10003137,2015-10-01,15,0


# Construção de Over90

In [34]:
# Cria coluna over90
df['over90'] = np.where(df['dias_atraso'] >= 90, 1, 0)

In [35]:
# Distribuição formatada
print("📊 Distribuição da coluna 'over90':")
dist = df['over90'].value_counts(normalize=True, dropna=False).mul(100).round(2)
display(dist.rename("Percentual (%)").to_frame())

# Exemplo de registros com over90 = 1
print("\n🔍 Exemplos de contratos com over90 = 1 (dias_atraso ≥ 90):")
cols_exibir = ["id_original", "id_contrato", "data_ref", "dias_atraso", "flag_acordo", "over90"]
display(df[df["over90"] == 1][cols_exibir].head())


📊 Distribuição da coluna 'over90':


,Percentual (%)
over90,
0,83.33
1,16.67



🔍 Exemplos de contratos com over90 = 1 (dias_atraso ≥ 90):


,id_original,id_contrato,data_ref,dias_atraso,flag_acordo,over90
5,10000000,10000000,2015-06-01,90,0,1
7,10000000,10000000,2015-08-01,90,0,1
8,10000000,10000000,2015-09-01,90,0,1
15,10000001,10000001,2015-05-01,90,1,1
23,10000002,10000002,2015-08-01,90,1,1


# Construção de Mau

In [36]:
df["mau"] = np.where((df["flag_acordo"] == 1) | (df["over90"] == 1), 1, 0)

In [37]:
# Exibe distribuição geral
print("📊 Distribuição da coluna 'mau':")
print((df["mau"].value_counts(normalize=True, dropna=False) * 100)
      .rename({1: "mau = 1", 0: "mau = 0"}).round(2))
print()

📊 Distribuição da coluna 'mau':
mau
mau = 0    82.09
mau = 1    17.91
Name: proportion, dtype: float64



In [38]:
# Amostras específicas

# 1. Garante exclusividade dos grupos
ex_ambos   = df[(df["mau"] == 1) & (df["flag_acordo"] == 1) & (df["over90"] == 1)].copy()
ex_acordo  = df[(df["mau"] == 1) & (df["flag_acordo"] == 1) & (df["over90"] == 0)].copy()
ex_over90  = df[(df["mau"] == 1) & (df["flag_acordo"] == 0) & (df["over90"] == 1)].copy()

# 2. Contagens absolutas
n_total = df["mau"].sum()
n_acordo = len(ex_acordo)
n_over90 = len(ex_over90)
n_ambos  = len(ex_ambos)

# 3. Percentuais
pct_acordo = 100 * n_acordo / n_total
pct_over90 = 100 * n_over90 / n_total
pct_ambos  = 100 * n_ambos  / n_total

# 4. Exibição de amostras
cols_exibir = ["id_original", "id_contrato", "data_ref", "dias_atraso", "flag_acordo", "over90", "mau"]

print("🔹 Exemplos de 'mau' causados apenas por acordo (flag_acordo = 1 e over90 = 0):")
display(ex_acordo.head(5)[cols_exibir])

print("🔹 Exemplos de 'mau' causados apenas por atraso >= 90 dias (over90 = 1 e flag_acordo = 0):")
display(ex_over90.head(5)[cols_exibir])

print("🔹 Exemplos de 'mau' causados por acordo e atraso >= 90 dias (ambos = 1):")
display(ex_ambos.head(5)[cols_exibir])

# 5. Sumário
print("\n📊 Distribuição dos motivos para 'mau' (total: {:,})".format(n_total))
print(f"• Apenas acordo ...............: {n_acordo:,} registros ({pct_acordo:.2f}%)")
print(f"• Apenas atraso >= 90 dias ....: {n_over90:,} registros ({pct_over90:.2f}%)")
print(f"• Ambos (acordo e atraso ≥ 90): {n_ambos:,} registros ({pct_ambos:.2f}%)")


🔹 Exemplos de 'mau' causados apenas por acordo (flag_acordo = 1 e over90 = 0):


,id_original,id_contrato,data_ref,dias_atraso,flag_acordo,over90,mau
10,10000000,10000000,2015-11-01,60,1,0,1
90,10000009,10000009,2016-02-01,60,1,0,1
99,10000010,10000010,2015-09-01,60,1,0,1
130,10000015,10000015,2015-04-01,60,1,0,1
163,10000017,10000017,2016-10-01,60,1,0,1


🔹 Exemplos de 'mau' causados apenas por atraso >= 90 dias (over90 = 1 e flag_acordo = 0):


,id_original,id_contrato,data_ref,dias_atraso,flag_acordo,over90,mau
5,10000000,10000000,2015-06-01,90,0,1,1
7,10000000,10000000,2015-08-01,90,0,1,1
8,10000000,10000000,2015-09-01,90,0,1,1
67,10000007,10000007,2015-01-01,180,0,1,1
68,10000007,10000007,2015-02-01,120,0,1,1


🔹 Exemplos de 'mau' causados por acordo e atraso >= 90 dias (ambos = 1):


,id_original,id_contrato,data_ref,dias_atraso,flag_acordo,over90,mau
15,10000001,10000001,2015-05-01,90,1,1,1
23,10000002,10000002,2015-08-01,90,1,1,1
24,10000003,10000003,2015-01-01,120,1,1,1
70,10000007,10000007,2015-04-01,90,1,1,1
101,10000011,10000011,2015-02-01,90,1,1,1



📊 Distribuição dos motivos para 'mau' (total: 20,633)
• Apenas acordo ...............: 1,431 registros (6.94%)
• Apenas atraso >= 90 dias ....: 16,510 registros (80.02%)
• Ambos (acordo e atraso ≥ 90): 2,692 registros (13.05%)


## Experimento de Correção de Reaging

In [116]:
class AjustadorDPDSemReaging:
    """
    Constrói um `dpd_adjusted` que previne reaging artificial após
    reestruturação/renegociação.

    Estratégia:
      - Quando detectarmos um rollover de contrato (mudança em id_contrato) ou
        flag_acordo=1 E o DPD bruto cair mais do que `drop_threshold`, tratamos
        isso como um possível reaging suspeito.
      - Carregamos adiante um deslocamento (offset) para que o DPD ajustado
        permaneça contínuo através do rollover.
      - Se ocorrer uma cura real (DPD bruto <= `cure_dpd_tolerance` por
        `cure_consecutive_periods` períodos consecutivos), zeramos o offset
        dali em diante.

    Parâmetros
    ----------
    dpd_col : str
        Coluna com os dias em atraso brutos (padrão 'dias_atraso').
    flag_acordo_col : str
        Coluna com a flag de reestruturação/renegociação (0/1).
    id_original_col : str
        Coluna que identifica a raiz/encadeamento do contrato.
    id_contrato_col : str
        Coluna que identifica o contrato vigente.
    date_col : str ou None
        Coluna de data opcional para ordenar e calcular deltas de tempo
        (checagens de envelhecimento diário).
    drop_threshold : int
        Queda mínima (em dias) para sinalizar reaging suspeito no rollover.
    cure_dpd_tolerance : int
        Limite de DPD para considerar estado “curado”.
    cure_consecutive_periods : int
        Número de períodos consecutivos no/abaixo do limite para aceitar cura.
    use_expected_aging : bool
        Se True e `date_col` for fornecida, comparamos a evolução do DPD bruto
        com o envelhecimento diário esperado.

    Retorno
    -------
    transform(df) -> DataFrame
        Retorna uma cópia de df com colunas adicionadas:
            - 'dpd_adjusted'
            - 'dpd_offset' (offset aplicado)
            - 'reaging_flag' (True onde reaging suspeito for detectado)
            - 'cure_flag' (True onde a cura for confirmada)
            - 'transition_flag' (True onde houve mudança de id_contrato)
    """

    def __init__(
        self,
        dpd_col: str = "dias_atraso",
        flag_acordo_col: str = "flag_acordo",
        id_original_col: str = "id_original",
        id_contrato_col: str = "id_contrato",
        date_col: str | None = None,
        drop_threshold: int = 15,
        cure_dpd_tolerance: int = 0,
        cure_consecutive_periods: int = 2,
        use_expected_aging: bool = True,
    ):
        self.dpd_col = dpd_col
        self.flag_acordo_col = flag_acordo_col
        self.id_original_col = id_original_col
        self.id_contrato_col = id_contrato_col
        self.date_col = date_col
        self.drop_threshold = int(drop_threshold)
        self.cure_dpd_tolerance = int(cure_dpd_tolerance)
        self.cure_consecutive_periods = int(cure_consecutive_periods)
        self.use_expected_aging = bool(use_expected_aging)

    def _prepare(self, df: pd.DataFrame) -> pd.DataFrame:
        dfc = df.copy()
        # Checagens básicas de existência de colunas obrigatórias
        for c in [self.dpd_col, self.flag_acordo_col, self.id_original_col, self.id_contrato_col]:
            if c not in dfc.columns:
                raise KeyError(f"Coluna obrigatória ausente: {c}")

        # Garantir tipos numéricos nas colunas relevantes
        dfc[self.dpd_col] = pd.to_numeric(dfc[self.dpd_col], errors="coerce").fillna(0)
        dfc[self.flag_acordo_col] = (
            pd.to_numeric(dfc[self.flag_acordo_col], errors="coerce").fillna(0).astype(int)
        )

        # Ordenação opcional por data; caso contrário, respeitar a ordem original por encadeamento
        if self.date_col is not None and self.date_col in dfc.columns:
            dfc[self.date_col] = pd.to_datetime(dfc[self.date_col], errors="coerce")
            sort_cols = [self.id_original_col, self.date_col, self.id_contrato_col]
        else:
            # Plano B: manter ordem de índice original dentro de id_original, depois por id_contrato
            dfc["_row_order__"] = np.arange(len(dfc), dtype=np.int64)
            sort_cols = [self.id_original_col, "_row_order__", self.id_contrato_col]

        # Salvar índice original para restaurar ao final
        dfc = (
            dfc.sort_values(sort_cols)
               .reset_index(drop=False)
               .rename(columns={"index": "_orig_index__"})
        )
        return dfc

    def transform(self, df: pd.DataFrame) -> pd.DataFrame:
        dfc = self._prepare(df)

        # Buffers de saída
        out_cols = {
            "dpd_adjusted": np.zeros(len(dfc), dtype=float),
            "dpd_offset": np.zeros(len(dfc), dtype=float),
            "reaging_flag": np.zeros(len(dfc), dtype=bool),
            "cure_flag": np.zeros(len(dfc), dtype=bool),
            "transition_flag": np.zeros(len(dfc), dtype=bool),
        }

        # Processar por encadeamento de contratos (id_original)
        for _, gidx in dfc.groupby(self.id_original_col, sort=False).groups.items():
            idx = list(gidx)
            offset = 0.0
            cure_streak = 0
            prev_raw = None
            prev_adj = None
            prev_contract = None
            prev_date = None

            for k, i in enumerate(idx):
                raw = float(dfc.at[i, self.dpd_col])
                acordo = int(dfc.at[i, self.flag_acordo_col])
                contrato = dfc.at[i, self.id_contrato_col]
                curr_date = None

                if self.date_col is not None and self.date_col in dfc.columns:
                    curr_date = dfc.at[i, self.date_col]

                # Detectar transição (novo contrato no encadeamento)
                transition = False
                if k > 0 and str(contrato) != str(prev_contract):
                    transition = True

                out_cols["transition_flag"][i] = transition

                # Envelhecimento diário esperado (opcional)
                expected_increase = 0.0
                if (
                    self.use_expected_aging
                    and curr_date is not None
                    and prev_date is not None
                    and prev_adj is not None
                ):
                    # DPD esperado hoje ~ prev_adj + dias decorridos (não menor que prev_adj)
                    try:
                        delta_days = max(0, (curr_date - prev_date).days)
                        expected_increase = float(delta_days)
                    except Exception:
                        expected_increase = 0.0

                # Reaging suspeito?
                #   condições:
                #   - transição de contrato OU flag de acordo
                #   - queda do DPD bruto grande vs nível de referência (prev_raw ou prev_adj + aging esperado)
                reaging = False
                if k > 0 and (transition or acordo == 1) and prev_raw is not None:
                    # Nível de referência anterior
                    reference_level = prev_raw
                    if self.use_expected_aging and prev_adj is not None:
                        reference_level = max(prev_raw, prev_adj + expected_increase)

                    drop = reference_level - raw
                    if drop >= self.drop_threshold:
                        reaging = True

                # Aplicar/manter offset conforme detecção de reaging
                if reaging:
                    # Garantir continuidade: fazer o ajustado “emendar” no nível anterior
                    # offset = (prev_adj ou prev_raw) - raw
                    base_prev = prev_adj if prev_adj is not None else prev_raw
                    if base_prev is None:
                        base_prev = 0.0
                    delta = base_prev - raw
                    offset += max(0.0, delta)
                    out_cols["reaging_flag"][i] = True
                else:
                    out_cols["reaging_flag"][i] = False

                # DPD ajustado
                adj = raw + offset
                if adj < 0:
                    adj = 0.0

                # Acompanhamento de cura (por padrão, usando DPD BRUTO; mude para adj se preferir)
                if raw <= self.cure_dpd_tolerance:
                    cure_streak += 1
                else:
                    cure_streak = 0

                # Se confirmou cura, zerar offset daqui em diante
                if cure_streak >= self.cure_consecutive_periods and offset != 0.0:
                    offset = 0.0
                    out_cols["cure_flag"][i] = True
                    # Recalcular DPD ajustado para esta linha após o reset
                    adj = raw + offset

                out_cols["dpd_adjusted"][i] = adj
                out_cols["dpd_offset"][i] = offset

                # Atualizar referências
                prev_raw = raw
                prev_adj = adj
                prev_contract = contrato
                prev_date = curr_date

        # Anexar saídas e restaurar a ordem original
        for c, arr in out_cols.items():
            dfc[c] = arr

        df_final = (
            dfc.sort_values("_orig_index__")
               .drop(columns=[col for col in ["_orig_index__", "_row_order__"] if col in dfc.columns])
               .set_index(df.index.__class__(df.index))  # manter tipo/valores do índice original
        )

        # Tipos finais convenientes
        df_final["dpd_adjusted"] = df_final["dpd_adjusted"].astype(int)
        df_final["dpd_offset"] = df_final["dpd_offset"].astype(int)
        return df_final


In [134]:
ajustador = AjustadorDPDSemReaging(
    dpd_col="dias_atraso",
    flag_acordo_col="flag_acordo",
    id_original_col="id_original",
    id_contrato_col="id_contrato",
    date_col="data_ref",             # ou None, se não houver
    drop_threshold=15,               # define o que é “queda abrupta”
    cure_dpd_tolerance=29,           # tolerância de cura (ex.: 0–5 dias)
    cure_consecutive_periods=3,      # quantos períodos seguidos ≤ tolerância para aceitar cura
    use_expected_aging=True          # usa diferença de dias entre linhas, se houver data
)

df_adj = ajustador.transform(df)

In [135]:
df_adj[df_adj['id_original']=='10000271']

,id_original,id_contrato,data_inicio_contrato,data_ref,safra,dias_atraso,data_evento,flag_acordo,over90,mau,dpd_adjusted,dpd_offset,reaging_flag,cure_flag,transition_flag
2650,10000271,10000271,2014-10-02,2015-01-01,201501,0,2015-05-01,0,0,0,0,0,False,False,False
2651,10000271,10000271,2014-10-02,2015-02-01,201501,0,2015-05-01,0,0,0,0,0,False,False,False
2652,10000271,10000271,2014-10-02,2015-03-01,201501,0,2015-05-01,0,0,0,0,0,False,False,False
2653,10000271,10000271,2014-10-02,2015-04-01,201501,0,2015-05-01,1,0,1,0,0,True,False,False
37814,10000271,10003137,2014-10-02,2015-05-01,201505,0,2016-06-01,0,0,0,0,0,True,False,True
37815,10000271,10003137,2014-10-02,2015-06-01,201505,0,2016-06-01,0,0,0,0,0,False,False,False
37816,10000271,10003137,2014-10-02,2015-07-01,201505,0,2016-06-01,0,0,0,0,0,False,False,False
37817,10000271,10003137,2014-10-02,2015-08-01,201505,0,2016-06-01,0,0,0,0,0,False,False,False
37818,10000271,10003137,2014-10-02,2015-09-01,201505,0,2016-06-01,0,0,0,0,0,False,False,False
37819,10000271,10003137,2014-10-02,2015-10-01,201505,15,2016-06-01,0,0,0,15,0,False,False,False


In [136]:
df_adj = df_adj[[
 'id_original',
 'id_contrato',
 'data_inicio_contrato',
 'data_ref',
 'safra',
 'dias_atraso',
 'data_evento',
 'flag_acordo',
 'over90',
 'mau',
 'dpd_adjusted'
]]

## Construção de Grupo Performing PD

In [ ]:
def mark_performing_pd(
    df: pd.DataFrame,
    id_col: str,
    date_col: str,
    dpd_col: str = "dias_atraso", #days past due
    gap_cure: int = 3, # verificando em 3 meses
    bad_col: str = "mau", # mau = over90 + flag_acordo
    output_col: str = "performing_pd",
    copy: bool = True,
    count_default_month_in_cure: bool = False,
    cure_dpd_tolerance: int = 0,
    cure_bad_intolerance: bool = True,
) -> pd.DataFrame:
    """
    Cria a coluna `performing_pd` (0/1) segundo as regras:
      - performing_pd = 0 em qualquer linha com mau == 1
      - Após um evento de 'mau == 1', a observação permanece como não-performada (0)
        até que:
          * o DPD zere (dpd == 0) e
          * existam 'gap_cure' períodos consecutivos com dpd == 0.
        A partir do período que completa essa janela de cura, volta a 1.

    Parâmetros
    ----------
    df : DataFrame
    id_col : str
        Identificador de cliente/contrato.
    date_col : str
        Coluna de referência temporal (ex.: safra). Pode ser datetime ou inteiro yyyymm.
    dpd_col : str, default "dias_atraso"
        Coluna de dias em atraso.
    gap_cure : int, default 3
        Janela de cura em *períodos consecutivos* (assume painel mensal).
    mau_col : str, default "mau"
        Coluna binária que marca evento de mau pagador.
    output_col : str, default "performing_pd"
        Nome da coluna de saída (0/1).
    copy : bool, default True
        Se True, trabalha numa cópia do df.

    Retorna
    -------
    DataFrame com a coluna `output_col` adicionada.

    Premissas
    ---------
    - O painel é (idealmente) mensal. Se houver buracos de meses, eles quebram a sequência
      consecutiva da cura de forma conservadora.
    - `dpd_col` é numérico (valores negativos/NaN tratados como 0 no critério de “zerado?”).
    - `mau_col` já existe (ex.: derivado de `over90` + `flag_acordo`).
    """
    # --- validations ---
    for c in (id_col, date_col, dpd_col, bad_col):
        if c not in df.columns:
            raise ValueError(f"Column '{c}' not found in df.")

    original_cols = list(df.columns)
    out = df.copy() if copy else df

    # --- month ordinal ---
    def _to_month_ordinal(series: pd.Series) -> np.ndarray:
        if is_datetime64_any_dtype(series):
            per = series.dt.to_period("M")
            return per.astype("period[M]").astype(int)
        if is_integer_dtype(series):
            y = series // 100
            m = series % 100
            return (y * 12 + (m - 1)).to_numpy()
        s_str = series.astype(str).str.strip()
        mask_6 = s_str.str.fullmatch(r"\d{6}", na=False)
        ord_arr = np.full(len(s_str), np.nan, dtype="float64")
        if mask_6.any():
            s6 = s_str[mask_6].astype(int)
            y = s6 // 100
            m = s6 % 100
            ord_arr[mask_6] = (y * 12 + (m - 1)).to_numpy()
        rest = ~mask_6
        if rest.any():
            dt = pd.to_datetime(s_str[rest], errors="coerce")
            per = dt.dt.to_period("M")
            ord_arr[rest] = per.astype("period[M]").astype("Int64").astype("float64")
        if np.isnan(ord_arr).any():
            raise ValueError(f"Could not parse some values in '{date_col}' to monthly periods.")
        return ord_arr.astype(int)

    out["_month_ord"] = _to_month_ordinal(out[date_col])

    # --- sanitize DPD and BAD ---
    dpd = pd.to_numeric(out[dpd_col], errors="coerce").fillna(0).clip(lower=0)
    out["_dpd_val"] = dpd.astype(int)  # keep numeric DPD for tolerance checks

    bad = pd.to_numeric(out[bad_col], errors="coerce").fillna(0).astype(int)
    out[bad_col] = (bad > 0).astype(int)

    # --- stable ordering + preallocate ---
    out["_orig_idx"] = np.arange(len(out))
    out.sort_values([id_col, "_month_ord", "_orig_idx"], inplace=True)
    out[output_col] = 0

    # --- group loop (months-based cure with DPD tolerance) ---
    for _, g_idx in out.groupby(id_col, sort=False).groups.items():
        g = out.loc[g_idx]

        b  = g[bad_col].to_numpy(dtype=int)
        d  = g["_dpd_val"].to_numpy(dtype=int)
        mo = g["_month_ord"].to_numpy(dtype=int)

        nonperf = np.zeros(len(g), dtype=bool)

        in_nonperf = False
        last_default_ord = None
        cure_start_ord = None  # first month (after default) with DPD <= tolerance

        for i in range(len(g)):
            if b[i] == 1:
                # Always non-performing on a default row
                nonperf[i] = True
                in_nonperf = True

                # First default observed => initialize boundaries
                if last_default_ord is None:
                    last_default_ord = mo[i]
                    cure_start_ord = mo[i] if (count_default_month_in_cure and d[i] <= cure_dpd_tolerance) else None
                else:
                    # Subsequent default while already curing
                    if cure_bad_intolerance:
                        last_default_ord = mo[i]
                        cure_start_ord = mo[i] if (count_default_month_in_cure and d[i] <= cure_dpd_tolerance) else None
                # go to next step (the rest of logic below still applies for the row)
            # If we're in non-performing phase (after a default), track the cure
            if in_nonperf:
                nonperf[i] = True  # default until cure completes
                if d[i] <= cure_dpd_tolerance:
                    # start cure if not started and strictly AFTER the default month
                    if cure_start_ord is None:
                        if (last_default_ord is None) or (mo[i] > last_default_ord) or \
                           (count_default_month_in_cure and mo[i] >= last_default_ord):
                            cure_start_ord = mo[i]
                    # check elapsed months (calendar-based)
                    if cure_start_ord is not None:
                        elapsed = mo[i] - cure_start_ord + 1
                        if elapsed >= gap_cure:
                            in_nonperf = False
                            nonperf[i] = False
                else:
                    # DPD above tolerance resets the cure window
                    cure_start_ord = None
            else:
                nonperf[i] = False  # performing

        performing = (~nonperf).astype(int)
        # Hard rule: same-row bad==1 forces 0
        performing[b == 1] = 0
        out.loc[g.index, output_col] = performing

    # --- restore order and cleanup ---
    out.sort_values("_orig_idx", inplace=True)
    out.drop(columns=["_orig_idx", "_month_ord", "_dpd_val"], inplace=True)

    # --- preserve all columns + append output at the end ---
    if output_col in original_cols:
        final_cols = original_cols
    else:
        final_cols = original_cols + [output_col]
    return out[final_cols]


In [140]:
df_pd = mark_performing_pd(
    df_adj,
    id_col="id_original",
    date_col="data_ref",
    dpd_col="dpd_adjusted",
    gap_cure=3,
    bad_col="mau",
    cure_dpd_tolerance=29,        # tolerancia de atraso para reiniciar o relógio de cura
    cure_bad_intolerance=True,   # novo mau durante a cura reinicia o relógio de cura
)

print(df_adj.shape)
print(df_pd.shape)

(115186, 11)
(115186, 12)


In [ ]:
# cure_dpd_tolerance 0 dias
#df_pd['performing_pd'].value_counts(normalize=True, dropna=False)*100

performing_pd
0    50.746619
1    49.253381
Name: proportion, dtype: float64

In [ ]:
# cure_dpd_tolerance 29 dias
#df_pd['performing_pd'].value_counts(normalize=True, dropna=False)*100

performing_pd
1    54.971958
0    45.028042
Name: proportion, dtype: float64

In [ ]:
# cure_dpd_tolerance 59 dias
#df_pd['performing_pd'].value_counts(normalize=True, dropna=False)*100

performing_pd
1    59.048843
0    40.951157
Name: proportion, dtype: float64

In [ ]:
# cure_dpd_tolerance 89 dias
#df_pd['performing_pd'].value_counts(normalize=True, dropna=False)*100

performing_pd
1    64.152762
0    35.847238
Name: proportion, dtype: float64

In [141]:
df_pd[df_pd['id_original']=='10000271']

,id_original,id_contrato,data_inicio_contrato,data_ref,safra,dias_atraso,data_evento,flag_acordo,over90,mau,dpd_adjusted,performing_pd
2650,10000271,10000271,2014-10-02,2015-01-01,201501,0,2015-05-01,0,0,0,0,1
2651,10000271,10000271,2014-10-02,2015-02-01,201501,0,2015-05-01,0,0,0,0,1
2652,10000271,10000271,2014-10-02,2015-03-01,201501,0,2015-05-01,0,0,0,0,1
2653,10000271,10000271,2014-10-02,2015-04-01,201501,0,2015-05-01,1,0,1,0,0
37814,10000271,10003137,2014-10-02,2015-05-01,201505,0,2016-06-01,0,0,0,0,0
37815,10000271,10003137,2014-10-02,2015-06-01,201505,0,2016-06-01,0,0,0,0,0
37816,10000271,10003137,2014-10-02,2015-07-01,201505,0,2016-06-01,0,0,0,0,1
37817,10000271,10003137,2014-10-02,2015-08-01,201505,0,2016-06-01,0,0,0,0,1
37818,10000271,10003137,2014-10-02,2015-09-01,201505,0,2016-06-01,0,0,0,0,1
37819,10000271,10003137,2014-10-02,2015-10-01,201505,15,2016-06-01,0,0,0,15,1


## Construção de Target_Bad_m12 (com flag_acordo embutida)

In [185]:
from pandas.api.types import is_datetime64_any_dtype, is_integer_dtype

class CalculadoraTargetBad:
    """
    Calcula targets binários a partir de uma flag de 'bad' (ex.: 'mau'),
    por contrato e ao longo de uma janela futura em períodos calendário.

    Parâmetros
    ----------
    date_col : str
        Coluna temporal. Aceita:
          - datetime64 (Timestamp)
          - inteiro no formato yyyymm (ex.: 202312)
          - string parseável para data (ex.: '2023-12' ou '2023-12-31')
    bad_col : str, default 'mau'
        Coluna binária com evento ruim (0/1).
    contract_col : str, default 'id_contrato'
        Identificador do contrato (agrupamento).
    modality : {'ever','over'}, default 'ever'
        - 'ever': 1 se existir ao menos um bad==1 na janela futura.
        - 'over': 1 se todos os períodos OBSERVADOS na janela futura forem bad==1.
                  (Períodos não observados não penalizam.)
    period : {'M','Q','Y'}, default 'M'
        Frequência calendário para a janela (Mensal, Trimestral, Anual).
    future_window : int, default 12
        Tamanho da janela futura em unidades de `period`.
    include_current : bool, default False
        Se True, inclui o período corrente no cálculo da janela; caso contrário,
        a janela começa no próximo período calendário.
    copy : bool, default True
        Se True, trabalha numa cópia do DataFrame.

    Saída
    -----
    transform(df) -> DataFrame
        Retorna o DataFrame com a coluna de target anexada:
        `target_bad_{modality}_{period}_{future_window}` (0/1).

    Observações
    -----------
    - A janela é baseada em distância calendário (não em contagem de linhas).
      Gaps de calendário não “quebram” a janela: se um bad acontecer em
      t + k (k <= future_window), ele é considerado, mesmo que meses intermediários
      não estejam presentes no painel.
    - Se houver múltiplas linhas no mesmo contrato e mesmo período calendário,
      a lógica agrega nesse período por `max(bad)` antes de avaliar a janela,
      evitando duplicidade de avaliação.
    """

    def __init__(
        self,
        date_col: str,
        bad_col: str = "mau",
        contract_col: str = "id_contrato",
        modality: str = "ever",
        period: str = "M",
        future_window: int = 12,
        include_current: bool = False,
        copy: bool = True,
    ):
        if modality not in {"ever", "over"}:
            raise ValueError("`modality` deve ser 'ever' ou 'over'.")
        if period not in {"M", "Q", "Y"}:
            raise ValueError("`period` deve ser 'M', 'Q' ou 'Y'.")
        if future_window <= 0:
            raise ValueError("`future_window` deve ser inteiro positivo.")

        self.date_col = date_col
        self.bad_col = bad_col
        self.contract_col = contract_col
        self.modality = modality
        self.period = period
        self.future_window = int(future_window)
        self.include_current = bool(include_current)
        self.copy = bool(copy)

    # --------------------------------------------------------------------- #
    # Helpers
    # --------------------------------------------------------------------- #
    def _to_period_ordinal(self, s: pd.Series) -> np.ndarray:
        """
        Converte `date_col` para um ordinal inteiro por frequência `self.period`.
        - Para datetime: usa .dt.to_period(self.period)
        - Para inteiro yyyymm: mapeia para período ('M') e depois converte para a
          frequência alvo ('M','Q','Y') via .asfreq
        - Para string: tenta parsear com to_datetime e procede como datetime
        """
        if is_datetime64_any_dtype(s):
            per = s.dt.to_period(self.period)
            return per.astype(f"period[{self.period}]").astype(int)

        if is_integer_dtype(s):
            # assume yyyymm
            y = (s // 100).astype(int)
            m = (s % 100).astype(int)
            # cria data no primeiro dia do mês
            dt = pd.to_datetime(dict(year=y, month=m, day=1), errors="coerce")
            per = dt.dt.to_period(self.period)
            return per.astype(f"period[{self.period}]").astype(int)

        # strings ou outros -> tentar parse
        s_str = s.astype(str).str.strip()
        dt = pd.to_datetime(s_str, errors="coerce")
        if dt.isna().any():
            raise ValueError(f"Não foi possível converter alguns valores de '{self.date_col}' para datas.")
        per = dt.dt.to_period(self.period)
        return per.astype(f"period[{self.period}]").astype(int)

    # --------------------------------------------------------------------- #
    # Principal
    # --------------------------------------------------------------------- #
    def transform(self, df: pd.DataFrame) -> pd.DataFrame:
        # validações
        for c in (self.date_col, self.bad_col, self.contract_col):
            if c not in df.columns:
                raise ValueError(f"Coluna '{c}' não encontrada no DataFrame.")

        out = df.copy() if self.copy else df

        # sanitizar bad -> binário 0/1
        bad = pd.to_numeric(out[self.bad_col], errors="coerce").fillna(0).astype(int)
        out[self.bad_col] = (bad > 0).astype(int)

        # ordinal por frequência
        ord_col = "_per_ord__"
        out[ord_col] = self._to_period_ordinal(out[self.date_col])

        # manter ordem original para restaurar
        idx_col = "_orig_idx__"
        out[idx_col] = np.arange(len(out))

        # coluna alvo
        target_col = f"target_bad_{self.modality}_{self.period}_{self.future_window}"
        out[target_col] = 0  # prealocar

        # processar por contrato
        for _, gidx in out.groupby(self.contract_col, sort=False).groups.items():
            g = out.loc[gidx, [ord_col, self.bad_col]]

            # agrega por período calendário: max(bad) por período
            cal = g.groupby(ord_col, as_index=False)[self.bad_col].max().sort_values(ord_col)
            ords = cal[ord_col].to_numpy(dtype=int)
            bads = cal[self.bad_col].to_numpy(dtype=int)

            if len(ords) == 0:
                continue

            # pré-cálculo para cada índice comprimido (um por período)
            # janela [left, right_excl)
            from bisect import bisect_right

            any_in_window = np.zeros(len(ords), dtype=bool)
            all_in_window = np.zeros(len(ords), dtype=bool)

            for r in range(len(ords)):
                start = r if self.include_current else r + 1
                if start > len(ords) - 1:
                    # não há período observado dentro da janela
                    any_in_window[r] = False
                    all_in_window[r] = False
                    continue

                end_ord = ords[r] + self.future_window
                right_excl = bisect_right(ords, end_ord)  # primeiro índice > end_ord

                window_slice = slice(start, right_excl)
                window_bads = bads[window_slice]

                if window_bads.size == 0:
                    any_in_window[r] = False
                    all_in_window[r] = False
                else:
                    any_in_window[r] = np.any(window_bads == 1)
                    all_in_window[r] = np.all(window_bads == 1)

            # mapeia resultado por período comprimido
            if self.modality == "ever":
                per_result = any_in_window
            else:  # 'over'
                per_result = all_in_window

            # espalha de volta para as linhas originais do grupo
            # (todas as linhas com o mesmo período recebem o mesmo target)
            per_to_val = dict(zip(ords, per_result.astype(int)))
            out.loc[gidx, target_col] = out.loc[gidx, ord_col].map(per_to_val).fillna(0).astype(int)

        # restaurar ordem original e limpar colunas auxiliares
        out = out.sort_values(idx_col).drop(columns=[idx_col, ord_col])

        # manter colunas originais + target ao final
        if target_col in df.columns:
            return out[df.columns]  # se por algum motivo já existia, preserva ordem original
        else:
            return out[df.columns.tolist() + [target_col]]


In [ ]:
# Target: “ever default” em 12 meses, por contrato, painel mensal
calc_ever_12m = CalculadoraTargetBad(
    date_col="data_ref",            # sua coluna temporal
    bad_col="mau",
    contract_col="id_contrato",
    modality="ever",
    period="M", #Q(trimestral ou quarter) Y (anual)
    future_window=12,
)

In [ ]:
df_targets = calc_ever_12m.transform(df_pd)

In [188]:
calc_over_12m = CalculadoraTargetBad(
    date_col="data_ref",
    bad_col="mau",
    contract_col="id_contrato",
    modality="over",
    period="M",
    future_window=12,
)

df_targets = calc_over_12m.transform(df_targets)

In [189]:
df_targets[df_targets['id_original']=='10000271']

,id_original,id_contrato,data_inicio_contrato,data_ref,safra,dias_atraso,data_evento,flag_acordo,over90,mau,dpd_adjusted,performing_pd,target_bad_ever_M_12,target_bad_over_M_12
2650,10000271,10000271,2014-10-02,2015-01-01,201501,0,2015-05-01,0,0,0,0,1,1,0
2651,10000271,10000271,2014-10-02,2015-02-01,201501,0,2015-05-01,0,0,0,0,1,1,0
2652,10000271,10000271,2014-10-02,2015-03-01,201501,0,2015-05-01,0,0,0,0,1,1,1
2653,10000271,10000271,2014-10-02,2015-04-01,201501,0,2015-05-01,1,0,1,0,0,0,0
37814,10000271,10003137,2014-10-02,2015-05-01,201505,0,2016-06-01,0,0,0,0,0,1,0
37815,10000271,10003137,2014-10-02,2015-06-01,201505,0,2016-06-01,0,0,0,0,0,1,0
37816,10000271,10003137,2014-10-02,2015-07-01,201505,0,2016-06-01,0,0,0,0,1,1,0
37817,10000271,10003137,2014-10-02,2015-08-01,201505,0,2016-06-01,0,0,0,0,1,1,0
37818,10000271,10003137,2014-10-02,2015-09-01,201505,0,2016-06-01,0,0,0,0,1,1,0
37819,10000271,10003137,2014-10-02,2015-10-01,201505,15,2016-06-01,0,0,0,15,1,1,0


In [190]:
df_targets['target_bad_ever_M_12'].value_counts(normalize=True, dropna=False)*100

target_bad_ever_M_12
0    52.560207
1    47.439793
Name: proportion, dtype: float64

In [191]:
df_targets[df_targets['performing_pd']==1]['target_bad_ever_M_12'].value_counts(normalize=True, dropna=False)*100

target_bad_ever_M_12
0    59.257738
1    40.742262
Name: proportion, dtype: float64

In [192]:
df_targets[df_targets['performing_pd']==1]['target_bad_over_M_12'].value_counts(normalize=True, dropna=False)*100

target_bad_over_M_12
0    96.51295
1     3.48705
Name: proportion, dtype: float64

In [148]:
# PASSO 1: Gerar o conjunto de dados robusto
# (10.000 contratos * 24 meses = 240.000 registros)
dados = pd.read_excel('gabarito_ever_over.xlsx')

# Mostra informações sobre o DataFrame gerado
print("\n--- Informações do DataFrame Gerado ---")
dados.info()
print("\n--- Primeiros Registros ---")
display(dados.head())
print("\n--- Últimos Registros ---")
display(dados.tail())


--- Informações do DataFrame Gerado ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   safra_ref                  36 non-null     int64
 1   id_contrato                36 non-null     int64
 2   atraso                     36 non-null     int64
 3   gabarito_over90            36 non-null     int64
 4   gabarito_over90m12         36 non-null     int64
 5   gabarito_ever90m12         36 non-null     int64
 6   flag_acordo                36 non-null     int64
 7   gabarito_bad90             36 non-null     int64
 8   gabarito_target_ever90m12  36 non-null     int64
 9   gabarito_target_over90m12  36 non-null     int64
dtypes: int64(10)
memory usage: 2.9 KB

--- Primeiros Registros ---


,safra_ref,id_contrato,atraso,gabarito_over90,gabarito_over90m12,gabarito_ever90m12,flag_acordo,gabarito_bad90,gabarito_target_ever90m12,gabarito_target_over90m12
0,202001,1000010,0,0,0,0,0,0,0,0
1,202002,1000010,0,0,0,0,0,0,1,1
2,202003,1000010,0,0,0,0,0,0,1,0
3,202004,1000010,15,0,1,1,0,0,1,1
4,202005,1000010,45,0,0,1,0,0,1,0



--- Últimos Registros ---


,safra_ref,id_contrato,atraso,gabarito_over90,gabarito_over90m12,gabarito_ever90m12,flag_acordo,gabarito_bad90,gabarito_target_ever90m12,gabarito_target_over90m12
31,202208,1000010,5,0,0,1,0,0,1,0
32,202209,1000010,15,0,0,1,0,0,1,0
33,202210,1000010,30,0,0,1,0,0,1,0
34,202211,1000010,60,0,0,1,0,0,1,0
35,202212,1000010,90,1,0,0,0,1,0,0


In [178]:
import plotly.graph_objects as go
from pandas.api.types import is_datetime64_any_dtype, is_integer_dtype

def plot_volume_e_percentual(
    df: pd.DataFrame,
    date_col: str,
    contract_col: str = "id_contrato",
    y2_col: str = "target",
    *,
    # Agregações
    volume_agg: str = "nunique",   # {'nunique', 'count'}
    y2_agg: str = "mean",          # {'mean', 'sum'}
    # Frequência / calendário
    freq: str | None = "M",        # {'M','Q','Y'} ou None para não reagrupar por calendário
    include_current_period: bool = True,  # se freq is not None, usar o timestamp do INÍCIO do período (True) ou do FIM (False)
    # Escala e formatação do eixo secundário
    percent_scale: str = "auto",   # {'auto','rate','pct'}
    # Aparência
    title: str | None = None,
    y1_name: str = "Qtd. Contratos",
    y2_name: str | None = None,
    y1_range: tuple | None = None,
    y2_range: tuple | None = None,
    bar_color: str = "silver",
    line_color: str = "#11BF5D",
    height: int = 520,
    show: bool = False,            # 🔧 evita gráfico duplicado em notebooks
):
    """
    Cria gráfico Plotly (graph_objects) com eixo X temporal (date_col),
    Y primário = volume de `contract_col` (barras),
    Y secundário = percentual/valor de `y2_col` (linha). A legenda fica
    abaixo do eixo X para não sobrepor o traçado.
    """
    # -------------------- validações --------------------
    if date_col not in df.columns:
        raise ValueError(f"Coluna temporal '{date_col}' não encontrada.")
    if contract_col not in df.columns:
        raise ValueError(f"Coluna de contrato '{contract_col}' não encontrada.")
    if y2_col not in df.columns:
        raise ValueError(f"Coluna '{y2_col}' não encontrada.")
    if volume_agg not in {"nunique", "count"}:
        raise ValueError("volume_agg deve ser 'nunique' ou 'count'.")
    if y2_agg not in {"mean", "sum"}:
        raise ValueError("y2_agg deve ser 'mean' ou 'sum'.")
    if percent_scale not in {"auto", "rate", "pct"}:
        raise ValueError("percent_scale deve ser 'auto', 'rate' ou 'pct'.")

    data = df.copy()

    # ---------------- normalizar coluna de data ----------------
    s = data[date_col]
    if is_datetime64_any_dtype(s):
        dt = s
    elif is_integer_dtype(s):
        # assume yyyymm
        y = (s // 100).astype(int)
        m = (s % 100).astype(int)
        dt = pd.to_datetime(dict(year=y, month=m, day=1), errors="coerce")
    else:
        dt = pd.to_datetime(s.astype(str), errors="coerce")
    if dt.isna().any():
        raise ValueError(f"Alguns valores de '{date_col}' não puderam ser convertidos para data.")
    data["_dt__"] = dt

    # -------------- chave de agrupamento temporal --------------
    if freq is None:
        data["_grp__"] = data["_dt__"]
        x_display = data["_grp__"].dt.strftime("%Y-%m-%d")
    else:
        per = data["_dt__"].dt.to_period(freq)
        grp_dt = per.dt.to_timestamp(how=("start" if include_current_period else "end"))
        data["_grp__"] = grp_dt
        if freq == "M":
            x_display = data["_grp__"].dt.strftime("%Y-%m")
        elif freq == "Q":
            x_display = per.astype(str)  # '2023Q1'
        elif freq == "Y":
            x_display = data["_grp__"].dt.strftime("%Y")
        else:
            x_display = data["_grp__"].astype(str)

    data["_x_str__"] = x_display

    # ------------------- sanitizar y2 -------------------
    y2_num = pd.to_numeric(data[y2_col], errors="coerce")
    if y2_agg == "sum":
        y2_num = y2_num.fillna(0)
    data["_y2__"] = y2_num

    # ------------------- agregações --------------------
    # Agrupar por rótulo exibido, garantindo ordenação cronológica
    # (os rótulos são ordenados por _grp__, não alfabeticamente)
    order = (
        data[["_x_str__", "_grp__"]]
        .drop_duplicates()
        .sort_values("_grp__")
    )
    gb = data.groupby("_x_str__", sort=False)

    vol = gb[contract_col].nunique() if volume_agg == "nunique" else gb[contract_col].count()
    y2_val = gb["_y2__"].mean() if y2_agg == "mean" else gb["_y2__"].sum()

    resumo = (
        pd.DataFrame({
            "_x_str__": vol.index,
            "volume": vol.values,
            "y2_val": y2_val.reindex(vol.index).values
        })
        .merge(order, on="_x_str__", how="left")
        .sort_values("_grp__")
        .reset_index(drop=True)
    )

    # ------------------- escala do Y2 -------------------
    if percent_scale == "rate":
        resumo["y2_plot"] = resumo["y2_val"] * 100.0
        y2_is_pct = True
    elif percent_scale == "pct":
        resumo["y2_plot"] = resumo["y2_val"]
        y2_is_pct = True
    else:
        vals = resumo["y2_val"].dropna().to_numpy()
        if vals.size and np.nanmax(vals) <= 1.0:
            resumo["y2_plot"] = resumo["y2_val"] * 100.0
            y2_is_pct = True
        else:
            resumo["y2_plot"] = resumo["y2_val"]
            y2_is_pct = False

    # ---------------- nomes / títulos -------------------
    if y2_name is None:
        y2_name = (f"% {y2_col}" if y2_is_pct else y2_col)
    if title is None:
        base = "Evolução de Volume e " + ("Percentual" if y2_is_pct else "Valor") + f" de '{y2_col}'"
        if freq:
            base += f" por { {'M':'Mês','Q':'Trimestre','Y':'Ano'}.get(freq,freq) }"
        title = base

    # ------------------- hovers ------------------------
    hover_bar = "<b>Volume:</b> %{y:,}<extra></extra>"
    hover_line = f"<b>{y2_name}:</b> " + ("%{y:.2f}%<extra></extra>" if y2_is_pct else "%{y:.2f}<extra></extra>")

    # ------------------- figura ------------------------
    fig = go.Figure()

    # Barras (Y1)
    fig.add_trace(go.Bar(
        x=resumo["_x_str__"],
        y=resumo["volume"],
        name=y1_name,
        marker_color=bar_color,
        yaxis="y1",
        hovertemplate=hover_bar
    ))

    # Linha (Y2)
    fig.add_trace(go.Scatter(
        x=resumo["_x_str__"],
        y=resumo["y2_plot"],
        name=y2_name,
        mode="lines+markers",
        line=dict(color=line_color, width=2.5),
        marker=dict(color=line_color, size=6),
        yaxis="y2",
        hovertemplate=hover_line
    ))

    # ------------------- layout ------------------------
    fig.update_layout(
        title=title,
        template="plotly_white",
        height=height,
        hovermode="x unified",
        # ✅ legenda abaixo do eixo X, centralizada
        legend=dict(
            orientation="h",
            yanchor="top",
            y=-0.27,          # ajuste fino conforme necessidade
            xanchor="center",
            x=0.5,
            bgcolor="rgba(0,0,0,0)"
        ),
        margin=dict(b=90),   # espaço para a legenda inferior
        xaxis=dict(
            title="Período" if freq else "Data",
            tickangle=-45,
            type="category",
            categoryorder="array",
            categoryarray=resumo["_x_str__"].tolist()
        ),
        yaxis=dict(
            title=y1_name,
            side="left",
            showgrid=False,
            range=y1_range
        ),
        yaxis2=dict(
            title=y2_name,
            side="right",
            overlaying="y",
            showgrid=False,
            tickformat=(".1f" if y2_is_pct else ".2f"),
            ticksuffix=("%" if y2_is_pct else ""),
            range=y2_range
        )
    )

    if show:
        fig.show()
        return None
    return fig


In [177]:
# 1) Mensal: volume = contratos únicos; y2 = média de um target binário (em %)
plot_volume_e_percentual(
    df_targets[df_targets['performing_pd']==1],
    date_col="data_ref",          # pode ser datetime ou 202312 etc.
    contract_col="id_contrato",
    y2_col="target_bad_ever_M_12",
    volume_agg="nunique",
    y2_agg="mean",
    freq="M",                        # 'M', 'Q' ou 'Y'; None para não reagrupar
    percent_scale="auto",            # detecta e converte para %
)

In [179]:
per = pd.to_datetime(df_targets['data_ref']).dt.to_period('M')
mask = (df_targets['performing_pd'] == 1) & (per <= pd.Period('2016-12', freq='M'))

plot_volume_e_percentual(
    df_targets[mask],
    date_col="data_ref",
    contract_col="id_contrato",
    y2_col="target_bad_ever_M_12",
    volume_agg="nunique",
    y2_agg="mean",
    freq="M",
    percent_scale="auto",
    show=False,
)

In [ ]:
# # Target: “over default” em 4 trimestres (todos os trimestres observados na janela têm bad=1)
# calc_over_4q = CalculadoraTargetBad(
#     date_col="competencia",
#     bad_col="mau",
#     contract_col="id_contrato",
#     modality="over",
#     period="Q",
#     future_window=4,
# )
# df_targets = calc_over_4q.transform(df_targets)


In [149]:
# Target: “ever default” em 12 meses, por contrato, painel mensal
calc_ever_12m = CalculadoraTargetBad(
    date_col="safra_ref",            # sua coluna temporal
    bad_col="gabarito_bad90",
    contract_col="id_contrato",
    modality="ever",
    period="M",
    future_window=12,
)

dados_targets = calc_ever_12m.transform(dados)

In [150]:
dados_targets

,safra_ref,id_contrato,atraso,gabarito_over90,gabarito_over90m12,gabarito_ever90m12,flag_acordo,gabarito_bad90,gabarito_target_ever90m12,gabarito_target_over90m12,target_bad_ever_M_12
0,202001,1000010,0,0,0,0,0,0,0,0,0
1,202002,1000010,0,0,0,0,0,0,1,1,1
2,202003,1000010,0,0,0,0,0,0,1,0,1
3,202004,1000010,15,0,1,1,0,0,1,1,1
4,202005,1000010,45,0,0,1,0,0,1,0,1
5,202006,1000010,75,0,0,1,0,0,1,0,1
6,202007,1000010,0,0,0,1,0,0,1,0,1
7,202008,1000010,0,0,0,1,0,0,1,0,1
8,202009,1000010,0,0,0,1,0,0,1,0,1
9,202010,1000010,0,0,0,1,0,0,1,0,1
